In [2]:
# Useful starting lines
%matplotlib inline
!pip install sklearn
import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [3]:
from sklearn import datasets

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [4]:
def hinge(x):
    '''
    x = np array 
    '''
    return np.maximum(0, 1-x)

def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    hinge_arg = np.multiply(y, X.dot(w))
    print(hinge_arg.shape)
    return np.sum(hinge(hinge_arg)) + lambda_/2*w.T.dot(w)


In [14]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    pred = (X.dot(w) > 0)*2 -1
    return ((pred==y)*1.0).mean()


## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [20]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    if (y_n*x_n.T.dot(w) >= 1):
        return lambda_*w
    else:
        return -y_n*x_n*num_examples + lambda_*w


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [21]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
            print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

    

sgd_for_svm_demo(y, X)

(569,)
iteration=0, cost=36846513.06466666
training accuracy = 0.37258347978910367
(569,)
iteration=10000, cost=535.4319149379232
training accuracy = 0.7803163444639719
(569,)
iteration=20000, cost=201.9607728331365
training accuracy = 0.9226713532513181
(569,)
iteration=30000, cost=175.01842471159765
training accuracy = 0.9138840070298769
(569,)
iteration=40000, cost=144.52068935301213
training accuracy = 0.9209138840070299
(569,)
iteration=50000, cost=139.3749151399091
training accuracy = 0.9209138840070299
(569,)
iteration=60000, cost=169.39138962906407
training accuracy = 0.9086115992970123
(569,)
iteration=70000, cost=133.0434802161852
training accuracy = 0.9209138840070299
(569,)
iteration=80000, cost=132.2909809080791
training accuracy = 0.9209138840070299
(569,)
iteration=90000, cost=149.72880007654354
training accuracy = 0.9121265377855887
(569,)
iteration=100000, cost=133.71746736839955
training accuracy = 0.9138840070298769
(569,)
iteration=110000, cost=127.28126306523964
tr

## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [24]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_examples)
    n: the coordinate to be updated
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    g = (1 - y_n * x_n.dot(w))

    if g != 0:
        alpha[n] = min(
            max(old_alpha_n + lambda_ * g / (x_n.T.dot(x_n)), 0.0),
            1.0)

        # compute the corresponding update on the primal vector w
        w += 1.0 / lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

In [25]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    return np.sum(alpha) - 1/2/lambda_*alpha.T.dot(np.diag(y)).dot(X).dot(X.T).dot(np.diag(y)).dot(alpha)

In [26]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

(569,)
iteration=0, primal:694.92983, dual:0.00613, gap:694.92370
(569,)
iteration=10000, primal:141.59107, dual:30.73336, gap:110.85770
(569,)
iteration=20000, primal:137.71690, dual:54.65594, gap:83.06096
(569,)
iteration=30000, primal:168.03794, dual:73.86968, gap:94.16826
(569,)
iteration=40000, primal:131.42123, dual:87.98745, gap:43.43378
(569,)
iteration=50000, primal:136.80576, dual:97.76828, gap:39.03748
(569,)
iteration=60000, primal:140.96554, dual:104.31351, gap:36.65202
(569,)
iteration=70000, primal:124.99602, dual:108.36250, gap:16.63352
(569,)
iteration=80000, primal:128.97640, dual:111.67348, gap:17.30292
(569,)
iteration=90000, primal:134.38212, dual:113.82744, gap:20.55467
(569,)
iteration=100000, primal:139.89398, dual:115.44571, gap:24.44827
(569,)
iteration=110000, primal:123.75090, dual:116.68625, gap:7.06465
(569,)
iteration=120000, primal:122.86854, dual:117.85916, gap:5.00938
(569,)
iteration=130000, primal:125.09847, dual:118.73784, gap:6.36063
(569,)
iterati